In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

C:\Users\porch\.conda\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# model = SentenceTransformer('sentence-transformers/nli-bert-base')
model = SentenceTransformer('bert-base-nli-mean-tokens')

C:\Users\porch\.conda\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\porch\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
results = []

for i in range(1,6):
    path = r"../data/us/newDataset/separate_5_folds_2/"
    train_df = pd.read_csv(path + '\\train_' + str(i) + '.csv')
    eval_df = pd.read_csv(path + '\\test_' + str(i) + '.csv')
    text_col=train_df.columns.values[1] 
    category_col=train_df.columns.values[2]

    x_eval = eval_df[text_col].values.tolist()
    y_eval = eval_df[category_col].values.tolist()

    x_train = train_df[text_col].values.tolist()
    y_train = train_df[category_col].values.tolist()
    
    training_embeddings = model.encode(x_train)
    testing_embeddings = model.encode(x_eval)
    print(i)
    yPred = []
    for i in testing_embeddings:
        _, index = findMaxVec2(i, training_embeddings)
        yPred.append(y_train[index[0][1]])
#     print(classification_report(y_eval, yPred))
    results.append(evaluation(y_eval, yPred))
    
CapR = CapP = CapF1 = HGP = HGR = HGF1 = SGP = SGR = SGF1 = TP = TR = TF1 = 0
for i in results:
    CapP += i['CapP']
    CapR += i['CapR']
    CapF1 += i['CapF1']
    HGP += i['HGP']
    HGR += i['HGR']
    HGF1 += i['HGF1']
    SGP += i['SGP']
    SGR += i['SGR']
    SGF1 += i['SGF1']
    TP += i['TP']
    TR += i['TR']
    TF1 += i['TF1']
print(CapP/len(results))
print(CapR/len(results))
print(CapF1/len(results))

print(HGP/len(results))
print(HGR/len(results))
print(HGF1/len(results))

print(SGP/len(results))
print(SGR/len(results))
print(SGF1/len(results))

print(TP/len(results))
print(TR/len(results))
print(TF1/len(results))   

1
2
3
4
5
0.8112252441444138
0.7766423357664234
0.7931026255196353
0.16499999999999998
0.175
0.16576923076923078
0.47864347776462024
0.4666666666666666
0.47094884945631216
0.2787719298245614
0.38888888888888884
0.32214513844948633


In [ ]:
# print(len(testing_embeddings[0]))
# for i in range(len(testing_embeddings) - 1):
#     print(len(testing_embeddings[i]))


In [6]:
def findMaxVec(sent, labels_embeddings):
    similarities = cosine_similarity(model.encode([sent]), labels_embeddings)
    max_vector = max(similarities[0])
    # return array because similarites is 2D array
    max_index = np.argwhere(similarities == max_vector)
    return max_vector, max_index

In [7]:
def findMaxVec2(testing_embeddings, training_embeddings):
    similarities = cosine_similarity([testing_embeddings], training_embeddings)
    max_vector = max(similarities[0])
    # return array because similarites is 2D array
    max_index = np.argwhere(similarities == max_vector)
    return max_vector, max_index

In [8]:
# print(findMaxVec2(testing_embeddings[0], training_embeddings))
# print(findMaxVec(x_eval[0], training_embeddings))

In [9]:
def evaluation(labels, preds, target_names = None):
    target_names = ['Capability', 'Hard-goal', 'Soft-goal', 'Task']
    metricReport = classification_report(labels, preds, zero_division=0, output_dict=True)
    return {
        'Accuracy': metricReport['accuracy'],
        'CapP': metricReport[target_names[0]]['precision'],
        'CapR': metricReport[target_names[0]]['recall'],              
        'CapF1': metricReport[target_names[0]]['f1-score'],
        'HGP': metricReport[target_names[1]]['precision'],
        'HGR': metricReport[target_names[1]]['recall'],
        'HGF1': metricReport[target_names[1]]['f1-score'],
        'SGP': metricReport[target_names[2]]['precision'],
        'SGR': metricReport[target_names[2]]['recall'],
        'SGF1': metricReport[target_names[2]]['f1-score'],
        'TP': metricReport[target_names[3]]['precision'],
        'TR': metricReport[target_names[3]]['recall'],
        'TF1': metricReport[target_names[3]]['f1-score'],
    }